# Lecture5 実習

ここでは、「マンガと学ぶデータビジュアライゼーション」掲載のデータを使って様々な可視化を実習します。

https://kakeami.github.io/viz-madb/index.html


```
データビジュアライゼーションとは，数値や文章などのデータに基づいた情報を，人間が理解しやすい形に視覚化する技術を指します．

このサイトは，文化庁のメディア芸術データベース・ラボ（MADB Lab）で公開されている四大少年誌（ 週刊少年サンデー， 週刊少年ジャンプ， 週刊少年チャンピオン， 週刊少年マガジン ）のデータを用いて，データビジュアライゼーションの学習を手助けすることを目指しています
```

# 1.棒グラフ

In [ ]:
#Colabならいらない
#!pip install plotly

In [ ]:
import itertools
import pandas as pd
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 前処理の結果，以下に分析対象ファイルが格納されていることを想定
#PATH_DATA = 'data/preprocess/out/episodes.csv'
# Jupyter Book用のPlotlyのrenderer
#RENDERER = 'plotly_mimetype+notebook'
RENDERER = "colab"

In [ ]:
def show_fig(fig):
    """Jupyter Bookでも表示可能なようRendererを指定"""
    fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))
    fig.show(renderer=RENDERER)

In [ ]:
def add_years_to_df(df, unit_years=10):
    """unit_years単位で区切ったyears列を追加"""
    df_new = df.copy()
    df_new['years'] = \
        pd.to_datetime(df['datePublished']).dt.year \
        // unit_years * unit_years
    df_new['years'] = df_new['years'].astype(str)
    return df_new

In [ ]:
def resample_df_by_cname_and_years(df):
    """cnameとyearsのすべての組み合わせが存在するように0埋め
    この処理を実施しないと作図時にX軸方向の順序が変わってしまう"""
    df_new = df.copy()
    yearss = df['years'].unique()
    cnames = df['cname'].unique()
    for cname, years in itertools.product(cnames, yearss):
        df_tmp = df_new[
            (df_new['cname'] == cname)&\
            (df_new['years'] == years)]
        if df_tmp.shape[0] == 0:
            s = pd.Series(
                {'cname': cname,
                 'years': years,
                 'weeks': 0,},
                index=df_tmp.columns)
            df_new = df_new.append(
                s, ignore_index=True)
    return df_new

In [ ]:
def resample_df_by_creator_and_years(df):
    """creatorとyearsのすべての組み合わせが存在するように0埋め
    この処理を実施しないと作図時にX軸方向の順序が変わってしまう"""
    df_new = df.copy()
    yearss = df['years'].unique()
    creators = df['creator'].unique()
    for creator, years in itertools.product(creators, yearss):
        df_tmp = df_new[
            (df_new['creator'] == creator)&\
            (df_new['years'] == years)]
        if df_tmp.shape[0] == 0:
            s = pd.Series(
                {'creator': creator,
                 'years': years,
                 'weeks': 0,},
                index=df_tmp.columns)
            df_new = df_new.append(
                s, ignore_index=True)
    return df_new

あらかじめ[前処理](https://kakeami.github.io/viz-madb/appendix/preprocess.html)に従って作成したデータを使います。

In [ ]:
#df = pd.read_csv(PATH_DATA)
df=pd.read_csv('https://raw.githubusercontent.com/miwamasa/DataScience2023/main/notebooks/data/preprocess/out/episodes.csv')

In [ ]:
df.head()

### 1.3.2. 作品別の掲載週数（上位20作品）
まずは，作品ごとの掲載週数を見てみましょう．

In [ ]:
df_plot = df.value_counts('cname').reset_index(name='weeks').head(20)
fig = px.bar(df_plot, x='cname', y='weeks', 
             title='作品別の掲載週数')
fig.update_xaxes(title='作品名')
fig.update_yaxes(title='掲載週数')
show_fig(fig)

### 1.3.3. 作品別・年代別の掲載週数（上位20作品）
では，上記の作品はいつ頃掲載されたものなのでしょうか？ここでは：

- 集合棒グラフ（グループ化された棒グラフ）
- 積上げ棒グラフ

を使って，作品別・年代別の合計掲載週を可視化します

In [ ]:
# dfに10年区切りの年代情報を追加
df = add_years_to_df(df)

In [ ]:
# プロット用に集計
df_plot = df.groupby('cname')['years'].value_counts().\
    reset_index(name='weeks')
# 連載週数上位10作品を抽出
cnames = list(df.value_counts('cname').head(20).index)
df_plot = df_plot[df_plot['cname'].isin(cnames)].\
    reset_index(drop=True)
# cname，yearsでアップサンプリング
df_plot = resample_df_by_cname_and_years(df_plot)

In [ ]:
# 合計連載週数で降順ソート
df_plot['order'] = df_plot['cname'].apply(
    lambda x: cnames.index(x))
df_plot = df_plot.sort_values(
    ['order', 'years'], ignore_index=True)

In [ ]:
# 作図
fig = px.bar(
    df_plot, x='cname', y='weeks', color='years',
    color_discrete_sequence= px.colors.diverging.Portland,
    barmode='group', 
    title='作品別・年代別の合計掲載週数（集合棒グラフ）')
fig.update_xaxes(title='作品名')
fig.update_yaxes(title='合計連載週数')
show_fig(fig)

冒頭の棒グラフを年代ごとに分割し，作品ごとに横に並べました．このようなグラフを集合棒グラフと呼びます．

長所：
- 各作品・各年代の絶対値を比較しやすい
  - 例：1970年代はダメおやじ，1980年代はこちら葛飾区亀有公園前派出所が代表的
- 各作品がどの年代に掲載されたか定性的にわかりやすい
  - 例：ダメおやじ等は1970-1980年代，MAJORは1990-2010年代に掲載された

課題：
- 年代の数に比例して凡例の数が増えてしまうため，全体的に棒が細くなり，視認性が悪くなる
- 年代をまたがった**合計掲載週数**の比較がしづらい

In [ ]:
# 作図
fig = px.bar(
    df_plot, x='cname', y='weeks', color='years',
    color_discrete_sequence= px.colors.diverging.Portland,
    barmode='stack', 
    title='作品別・年代別の合計連載週数（積上げ棒グラフ）')
fig.update_xaxes(title='作品名')
fig.update_yaxes(title='合計連載週数')
show_fig(fig)

同じ情報を積上げ棒グラフで可視化したものです． 積上げ棒グラフは，年代ごとの掲載数を横に並べるのではなく，縦に積上げていることにご注意ください．

積上げ棒グラフの長所：

- 各作品の年代ごとの比率を比較しやすい
- 各作品の合計掲載週を比較しやすい

積上げ棒グラフの短所は：
- 各作品・各年代の絶対値を比較しづらい


### 1.3.4. 作家別の掲載週数（上位20名）
同様に，作家別に掲載週数を可視化してみましょう．

In [ ]:
df_plot = df.value_counts('creator').reset_index(name='weeks').head(20)
fig = px.bar(df_plot, x='creator', y='weeks', title='作者別の掲載週数')
fig.update_xaxes(title='作家名')
fig.update_yaxes(title='掲載週数')
show_fig(fig)

### 1.3.5. 作家別・年代別の掲載週数（上位20名）

In [ ]:
# 10年単位で区切ったyearsを追加
df = add_years_to_df(df)

In [ ]:
# プロット用に集計
df_plot = df.groupby('creator')['years'].value_counts().\
    reset_index(name='weeks')
# 連載週刊上位20作品を抽出
creators = list(df.value_counts('creator').head(20).index)
df_plot = df_plot[df_plot['creator'].isin(creators)].\
    reset_index(drop=True)
# creator，yearsでアップサンプリング
df_plot = resample_df_by_creator_and_years(df_plot)

In [ ]:
# 合計連載週数で降順ソート
df_plot['order'] = df_plot['creator'].apply(
    lambda x: creators.index(x))
df_plot = df_plot.sort_values(
    ['order', 'years'], ignore_index=True)

In [ ]:
# 作図
fig = px.bar(
    df_plot, x='creator', y='weeks', color='years',
    color_discrete_sequence= px.colors.diverging.Portland,
    barmode='group', title='作家別・年代別の掲載週数')
fig.update_xaxes(title='作家名')
fig.update_yaxes(title='掲載週数')
show_fig(fig)

In [ ]:
# 作図
fig = px.bar(
    df_plot, x='creator', y='weeks', color='years',
    color_discrete_sequence= px.colors.diverging.Portland,
    barmode='stack', title='作家別・年代別の掲載週数')
fig.update_xaxes(title='作家名')
fig.update_yaxes(title='掲載週数')
show_fig(fig)

## 1.4. 練習問題
1. 掲載週（datePublished）数ではなく，作品（cname）数が多い作家を可視化してみましょう．掲載週数と比較して言えることはありますか？
2. 年代別・作品数別に積上げ棒グラフを作成して，作家毎の特徴を考察してみましょう